In [ ]:
import torch
import os
import gzip
from glob import glob
import shutil
from tqdm import tqdm
import dicom2nifti
import numpy as np
import nibabel as nib
from monai.transforms import(
    Compose,
    AddChanneld,
    LoadImaged,
    Resized,
    ToTensord,
    Spacingd,
    Orientationd,
    ScaleIntensityRanged,
    CropForegroundd,
)
from monai.data import DataLoader, Dataset, CacheDataset
from monai.utils import set_determinism

In [2]:
def extract_gz_files(directory):
    """
    Extracts the contents of all .gz files in a directory and saves them in the same directory.
    
    Parameters:
    directory (str): Path to the directory containing the .gz files.
    """
    for filename in os.listdir(directory):
        if filename.endswith("nii.gz"):
            file_path = os.path.join(directory, filename)
            with gzip.open(file_path, "rb") as gz_file:
                uncompressed_data = gz_file.read()
            new_filename = filename[:-3]  # Remove the .gz extension
            new_file_path = os.path.join(directory, new_filename)
            with open(new_file_path, "wb") as new_file:
                new_file.write(uncompressed_data)
            os.remove(file_path)  # Delete the original .gz file


In [3]:
path1 = '/media/david/Elements/dev/DL/Monai-AI-Healthcare-Imaging/data/Task03_Liver/Task03_Liver/imagesTr'
path2 = '/media/david/Elements/dev/DL/Monai-AI-Healthcare-Imaging/data/Task03_Liver/Task03_Liver/imagesTs' 

In [3]:
os.listdir()

['.ipynb_checkpoints',
 'Computer-Vision-Healthcare',
 'data',
 'Preprocessing data.ipynb',
 'sources.txt']

In [6]:
paths = os.getcwd()
path_images_tr = os.path.join(paths, '', 'data/Task03_Liver/Task03_Liver/imagesTr')
path_dicom_files = os.path.join(paths, '', 'data/Task03_Liver/Task03_Liver/dicom_files')
path_dicom_groups = os.path.join(paths, '', 'data/Task03_Liver/Task03_Liver/dicom_groups')

In [7]:
# exemple
path_dicom_files

'/media/david/Elements/dev/DL/Monai-AI-Healthcare-Imaging/data/Task03_Liver/Task03_Liver/dicom_files'

## Labels

In [8]:
input_path_labels = os.path.join(path_dicom_file, '', 'labels')
input_path_labels

'/media/david/Elements/dev/DL/Monai-AI-Healthcare-Imaging/data/Task03_Liver/Task03_Liver/dicom_files/labels'

In [9]:
output_path_labels = os.path.join(path_dicom_groups, '', 'labels')
output_path_labels

'/media/david/Elements/dev/DL/Monai-AI-Healthcare-Imaging/data/Task03_Liver/Task03_Liver/dicom_groups/labels'

### Creation d'un groupe de 64 slices

On va convertir les 64 slices en 1 seul fichier nfti

In [4]:
from glob import glob
import shutil

In [11]:
for patient in glob(input_path_labels + '/*'):
    print(patient)

/media/david/Elements/dev/DL/Monai-AI-Healthcare-Imaging/data/Task03_Liver/Task03_Liver/dicom_files/labels/liver_0
/media/david/Elements/dev/DL/Monai-AI-Healthcare-Imaging/data/Task03_Liver/Task03_Liver/dicom_files/labels/liver_1


In [13]:
patient_list = glob(input_path_labels + '/*')
patient_list[0]

'/media/david/Elements/dev/DL/Monai-AI-Healthcare-Imaging/data/Task03_Liver/Task03_Liver/dicom_files/labels/liver_0'

* `os.path.basename()` : split au dernier '/' et récupère le nom du dossier ou du fichier 
* `os.path.normpath()` : supprime les sequences '.' et '..' inutiles dans le chemin  

In [18]:
groups_number = 64

for patient in glob(input_path_labels + '/*'):
    # patient_name = liver_00 (pour le 1er patient, ect...)
    patient_name = os.path.basename(os.path.normpath(patient))
    # glob(patient + '/*') crée une liste de tous les fichiers du liver_00 (pour le 1er patient)
    # on va récupérer le nombre total de fichier qu'on divise par le nombre de patients par groupe 
    # souhaités (dans notre cas, on veut 64 patients par groupe)
    # Attention : on ne prend que les groupes entier de 64 personnes
    # ex: si un groupe de patients est de 80 personnes, nous n'aurons qu'un seul groupe de 64 personnes
    # cad qu'on ne garde pas la partie non entière de 24 personnes (pour un problème d'accuracy en DL)
    number_folders = int(len(glob(patient + '/*')) / groups_number)
    
    # on va créer autant de sous-groupe de 64 patients que le nombre de number_folders
    for i in range(number_folders):
        # os.path.join(path, patient_name+'_' + str(i)) ==> path/patient_name_i
        output_path_name = os.path.join(output_path_labels, patient_name + f'_{str(i)}')
        # os.mkdir(folder_name) : va créer pour chaque folder_name un folder à son nom
        os.mkdir(output_path_name)
        
        all_patient = glob(patient + '/*')
        # pour chaque patient dans la liste all_patient
        for i, file in enumerate(all_patient):
            # si le nombre de fichier de patient est égale à 64, on stop la loop
            if i == groups_number :
                break
            # shutil.move(file, new_path) : on bouge le file vers le new_path
            shutil.move(file, output_path_name)

On va créer une fonction qui va creer 64 slices pour les images

## Images

In [20]:
input_path_images = os.path.join(path_dicom_file, '', 'images')
input_path_images

'/media/david/Elements/dev/DL/Monai-AI-Healthcare-Imaging/data/Task03_Liver/Task03_Liver/dicom_files/images'

In [21]:
output_path_images = os.path.join(path_dicom_groups, '', 'images')
output_path_images

'/media/david/Elements/dev/DL/Monai-AI-Healthcare-Imaging/data/Task03_Liver/Task03_Liver/dicom_groups/images'

In [50]:
def create_slices(input_path, output_path, nb_slices, source):
    input_path = os.path.join(input_path, '', source)
    output_path = os.path.join(output_path, '', source)
    
    for patient in glob(input_path + '/*'):
        patient_name = os.path.basename(os.path.normpath(patient))
        number_folders = int(len(glob(patient + '/*')) / nb_slices)
        
        for i in range(number_folders):
            output_path_name = os.path.join(output_path, patient_name + f'_{str(i)}')
            os.mkdir(output_path_name)
            all_patient = glob(patient + '/*')
            
            for i, file in enumerate(all_patient):
                if i == nb_slices :
                    break
                    
                shutil.move(file, output_path_name)

In [51]:
create_slices(input_path=path_dicom_file, output_path=path_dicom_groups, nb_slices=64, source='images')

## Convertir les fichiers dicom en nifties 

In [3]:
import dicom2nifti as d2n

In [27]:
all_input_path_images = os.path.join(path_dicom_groups, 'images/*')
all_input_path_labels = os.path.join(path_dicom_groups, 'labels/*')

list_images = glob(all_input_path_images)
list_labels = glob(all_input_path_labels)

In [28]:
list_images

['/media/david/Elements/dev/DL/Monai-AI-Healthcare-Imaging/data/Task03_Liver/Task03_Liver/dicom_groups/images/liver_0_0',
 '/media/david/Elements/dev/DL/Monai-AI-Healthcare-Imaging/data/Task03_Liver/Task03_Liver/dicom_groups/images/liver_1_0']

In [29]:
output_path_nifty_images = os.path.join(output_path_images[:-20], 'nifti_files/images')
output_path_nifty_images

'/media/david/Elements/dev/DL/Monai-AI-Healthcare-Imaging/data/Task03_Liver/Task03_Liver/nifti_files/images'

In [30]:
output_path_nifty_labels = os.path.join(output_path_labels[:-20], 'nifti_files/labels')
output_path_nifty_labels

'/media/david/Elements/dev/DL/Monai-AI-Healthcare-Imaging/data/Task03_Liver/Task03_Liver/nifti_files/labels'

In [35]:
for patient in list_images:
    # on récupère seulement le nom du patient (ex: 'liver_00')
    patient_name = os.path.basename(os.path.normpath(patient))
    # dicom2nifti.dicom_series_to_nifti(allpath_to_series_file, allpath_to_nifti_file) 
    # convertie une series en nifti (si on veut compresser le fichier : patient_name + '.nii.gz')
    d2n.dicom_series_to_nifti(patient, os.path.join(output_path_nifty_images, patient_name + '.nii'))

In [34]:
for patient in list_labels:
    # on récupère seulement le nom du patient (ex: 'liver_00')
    patient_name = os.path.basename(os.path.normpath(patient))
    # dicom2nifti.dicom_series_to_nifti(allpath_to_series_file, allpath_to_nifti_file) 
    # convertie une series en nifti (si on veut compresser le fichier : patient_name + '.nii.gz')
    d2n.dicom_series_to_nifti(patient, os.path.join(output_path_nifty_labels, patient_name + '.nii'))

## Find empty

In [5]:
o_n_i = '/media/david/Elements/dev/DL/Monai-AI-Healthcare-Imaging/data/Task03_Liver/Task03_Liver/nifti_files/images'
o_n_l = '/media/david/Elements/dev/DL/Monai-AI-Healthcare-Imaging/data/Task03_Liver/Task03_Liver/nifti_files/labels'

In [7]:
input_nifti_image_0 = os.path.join(o_n_i, 'liver_0_0.nii')
input_nifti_image_0

'/media/david/Elements/dev/DL/Monai-AI-Healthcare-Imaging/data/Task03_Liver/Task03_Liver/nifti_files/images/liver_0_0.nii'

In [ ]:
nifti_file = nib.load(input_nifti_image_0)
fdata = nifti_file.get_fdata()